In [89]:
import pandas as pd
import numpy as np
import mysql.connector
from openai import OpenAI
import os
import re
import json 
from dotenv import load_dotenv
load_dotenv()

True

In [83]:
# ChatGPTを使うのでAPIキーを設定
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gpt_client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [84]:
# ChatGPTを使って意味のある文章を生成する
def get_gpt_result(prompt):
    response = gpt_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


In [85]:
# DB接続処理
# MySQLに接続
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="foodb"
)
cursor = connection.cursor()


In [97]:
# トークン化して保存する
def tokenize_save(key_name, sentences):
    # NumPyの配列に変換
    tokens = []
    for sentence in sentences:
        # 小文字化し、特殊文字や句読点を削除してトークン化
        words = re.findall(r'\b\w+\b', sentence.lower())
        tokens.append(words)
        
    np_sentences = np.array(tokens, dtype=object)

    # NumPyの配列をdumpして保存
    np.save(f"../datas/word2_vec/{key_name}.npy", np_sentences)
    with open(f"../datas/word2_vec/{key_name}.txt", 'w') as f:
        json.dump(tokens, f)

##### 食品グループ

In [87]:
# 食品グループとサブグループの取得
food_groups = []
query = "select food_group, food_subgroup from foods where food_group is not null group by food_group, food_subgroup"
cursor.execute(query)

# 1行ずつ結果を取得
for row in cursor:
    sentences = f"{row[0].lower()} include {row[1].lower()}"
    food_groups.append(sentences)
    
    prompt = f"Generate a sentence that explains the relationship between {row[0]} and {row[1]}. For example: '{row[0]} include {row[1]} and are commonly used in various dishes for flavor.'"
    gpt_scentense = get_gpt_result(prompt)
    food_groups.append(gpt_scentense)


In [98]:
# 食品グループのデータを保存
tokenize_save("food_groups", food_groups)

#### 食品データを保存

In [ ]:
# 食品データを保存
foods = []
query = "select food_group, food_subgroup from foods where food_group is not null group by food_group, food_subgroup"
cursor.execute(query)

# 1行ずつ結果を取得
for row in cursor:
    sentences = f"{row[0].lower()} include {row[1].lower()}"
    food_groups.append(sentences)
    
    prompt = f"Generate a sentence that explains the relationship between {row[0]} and {row[1]}. For example: '{row[0]} include {row[1]} and are commonly used in various dishes for flavor.'"
    gpt_scentense = get_gpt_result(prompt)
    food_groups.append(gpt_scentense)

In [ ]:
# 食品グループのデータを保存
tokenize_save("foods", foods)

### 一般レシピのデータを保存